In [277]:
import os
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier

### Basic Logistic Regression

Use Michelle's method for importing data since I can't do it correctly:

In [2]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [3]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [4]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [50]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [51]:
train_X2 = np.array(train_X2)
valid_X2 = np.array(valid_X2)
train_y2 = np.array(train_y2)
valid_y2 = np.array(valid_y2)
test_X2 = np.array(test_X2)

Fit the model:

In [7]:
logistic = model.LogisticRegression()
logistic.fit(train_X2, train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Print the classification error on the training set:

In [8]:
class_error(logistic.predict(train_X2), train_y2)

0.118625

Print the classification error on the validation set:

In [9]:
class_error(logistic.predict(valid_X2), valid_y2)

0.16025

Logistic regression works better than a random forest. Now, we want to combine the training and validation sets again so that we can train our model on all the data to prevent overfitting. We will then use this to predict on the test set:

In [20]:
all_X = np.concatenate((train_X2, valid_X2))
all_y = np.concatenate((train_y2, valid_y2))

In [11]:
logistic = model.LogisticRegression()
logistic.fit(all_X, all_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

A function for writing to the text file for the submission:

In [82]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

In [13]:
writeToText(logistic.predict(test_X2), 'logistic.txt')

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [14]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

Fit the model using the training data:

In [18]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(train_X2, train_labels), train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
class_error(logistic.predict(TFIDF(valid_X2, train_labels)), valid_y2)

0.15375

This model has a lower classification error than the basic regression. We will now undergo the same procedure as in the last section to get the test predictions:

In [20]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(all_X, train_labels), all_y)
writeToText(logistic.predict(TFIDF(test_X2, train_labels)), 'logisticTFIDF.txt')

### GridSearch

In [34]:
clf = SGDClassifier(loss='hinge', penalty='l2', random_state=42).fit(train_X2, train_y2)
parameters = {'alpha': (1e-5, 1e-6)}
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(TFIDF(train_X2, train_labels), train_y2)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/anaconda/lib/python3.6/site-packages/skle

In [35]:
gs_clf.best_params_

{'alpha': 1e-05}

clf.get_params()

### Naive Bayes

In [75]:
nb = MultinomialNB(alpha=50).fit(TFIDF(train_X2, train_labels), train_y2)

In [33]:
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [76]:
class_error(nb.predict(TFIDF(valid_X2, train_labels)), valid_y2)

0.17

### SVM

In [275]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(TFIDF(train_X2, train_labels), train_y2)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [276]:
class_error(sgd.predict(valid_TFIDF), valid_y2)

0.15325

In [26]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)
sgd.fit(TFIDF(all_X, train_labels), all_y)
writeToText(sgd.predict(TFIDF(test_X2, train_labels)), 'SGD_TFIDF.txt')

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### Blend the based on equal weights for each one:

In [83]:
valid_TFIDF = TFIDF(valid_X2, train_labels)
train_TFIDF = TFIDF(train_X2, train_labels)
all_TFIDF = TFIDF(all_X, train_labels)
test_TFIDF = TFIDF(test_X2, train_labels)

In [78]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

pred1 = logistic.predict(valid_TFIDF)
pred2 = sgd.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [79]:
pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

In [80]:
class_error(pred, valid_y2)

0.15025

In [84]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

pred1 = logistic.predict(test_TFIDF)
pred2 = sgd.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)

pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

writeToText(pred, 'Blend_TFIDF.txt')

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### BernoulliNB

For this, we need binary features.

In [38]:
train_X2[train_X2 > 1] = 1
valid_X2[valid_X2 > 1] = 1
all_X[all_X > 1] = 1

In [54]:
bnb = BernoulliNB(alpha=1.0e-10)
bnb.fit(train_X2, train_y2)
class_error(bnb.predict(valid_X2), valid_y2)

0.1795

### Probability Blend

Each model outputs can output a probability rather than a 0 or 1. We should use these probabilities when classifying the points.

In [85]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [87]:
nb.predict_proba(valid_TFIDF)

array([[ 0.37797752,  0.62202248],
       [ 0.65392058,  0.34607942],
       [ 0.48520922,  0.51479078],
       ..., 
       [ 0.60520652,  0.39479348],
       [ 0.49716815,  0.50283185],
       [ 0.41540171,  0.58459829]])

In [89]:
sgd.decision_function(valid_TFIDF)

array([ 1.23658711, -1.94103518,  0.84981656, ..., -1.26886093,
       -0.86700456,  1.0053024 ])

In [90]:
valid_y2

array([1, 0, 1, ..., 0, 0, 1])

In [91]:
logistic.decision_function(valid_TFIDF)

array([ 1.84006047, -2.53061288,  0.62282974, ..., -1.99477366,
       -1.02389807,  1.59835106])

sgd and logistic have decision functions, whereas bayes has a probability. Since sgd and logistic are better models, we will look at when the magnitude of their confidence scores is greater than 1. If they both have magnitudes greater than 1 but opposite signs, or if the magnitude for both is less than 1, we get the majority vote from all 3 models.

In [96]:
def tieBreak(i, pred1, pred2, pred3):
    winner = 0
    if pred1[i] == pred2[i]:
        winner = pred1[i]
    elif pred1[i] == pred3[i]:
        winner = pred1[i]
    else:
        winner = pred2[i]
    return winner

In [239]:
sgd_confidence = sgd.decision_function(valid_TFIDF)
log_confidence = logistic.decision_function(valid_TFIDF)
pred1 = sgd.predict(valid_TFIDF)
pred2 = logistic.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(sgd_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) > 1:
        if (log_confidence[i] > 0 and sgd_confidence[i] > 0) or (log_confidence[i] < 0 and sgd_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred1, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred1, pred2, pred3))


In [184]:
class_error(pred_blend, valid_y2)

0.15025

### Linear SVC

In [181]:
lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(train_TFIDF, train_y2)

LinearSVC(C=2, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=0, tol=0.0001, verbose=0)

In [182]:
class_error(lin.predict(valid_TFIDF), valid_y2)

0.15475

### Ridge

In [237]:
ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(train_TFIDF, train_y2)

RidgeClassifier(alpha=15, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [238]:
class_error(ridge.predict(valid_TFIDF), valid_y2)

0.1525

### New Weighted Blends

In [243]:
def tieBreak2(i, pred1, pred2, pred3, pred4, pred5):
    preds = [pred1, pred2, pred3, pred4, pred5]
    zeros = 0
    for element in preds:
        if element[i] == 0:
            zeros += 1
    if zeros > 2:
        return 0
    else:
        return 1

In [254]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(train_TFIDF, train_y2)

ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

sgd_confidence = sgd.decision_function(valid_TFIDF)
log_confidence = logistic.decision_function(valid_TFIDF)
lin_confidence = lin.decision_function(valid_TFIDF)
ridge_confidence = ridge.decision_function(valid_TFIDF)

pred1 = sgd.predict(valid_TFIDF)
pred2 = logistic.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)
pred4 = lin.predict(valid_TFIDF)
pred5 = ridge.predict(valid_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(lin_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) > 1:
        if (log_confidence[i] > 0 and lin_confidence[i] > 0) or (log_confidence[i] < 0 and lin_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred4, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred4, pred2, pred3))

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [255]:
class_error(pred_blend, valid_y2)

0.149

It seems that a combination of linearSVC and LogisticRegression with Naive Bayes as the tie break works the best.

In [304]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(all_TFIDF, all_y)

ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

sgd_confidence = sgd.decision_function(test_TFIDF)
log_confidence = logistic.decision_function(test_TFIDF)
lin_confidence = lin.decision_function(test_TFIDF)
ridge_confidence = ridge.decision_function(test_TFIDF)

pred1 = sgd.predict(test_TFIDF)
pred2 = logistic.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)
pred4 = lin.predict(test_TFIDF)
pred5 = ridge.predict(test_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(lin_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) > 1:
        if (log_confidence[i] > 0 and lin_confidence[i] > 0) or (log_confidence[i] < 0 and lin_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred4, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred4, pred2, pred3))

/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [306]:
writeToText(pred_blend, 'Blend_TFIDF_Log_Lin.txt')

### K Neighbors Classifier

In [302]:
neigh = KNeighborsClassifier(n_neighbors=1000)
neigh.fit(train_TFIDF, train_y2) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1000, p=2,
           weights='uniform')

In [303]:
class_error(neigh.predict(valid_TFIDF), valid_y2)

0.181